In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import os
# load and display an image with Matplotlib
from matplotlib import image
from matplotlib import pyplot
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from numpy import asarray
import os

ModuleNotFoundError: No module named 'tensorflow'

# Task 1 - Getting the data in

# ift. opdelingen af test,train og validering har jeg simpelthen bare taget det manuelt fra den store mappe og opdelt det i 20% test og 20% af train til validering
# Dette betyder at vi fra det originale data har:
# test: 520 billeder, 260 af hver
# train: 1664 billeder, 832 af hver
# Validation: 416, 206 af hver

In [2]:
# goes through the folder containing the data and adds them to a list, generates labels and add data to validate labels against

def load_image_function(path):

    images = [] # empty list placeholder
    labels = [] # empty list placeholder
    animal_checker = 'cat' # used to create labels
    container = [] # container to validate correct labels

    for filename in os.listdir(path):

      container.append(filename) # add filename to container

      if animal_checker in filename:
        labels.append('cat') # check if cat is in filename. If so append cat to label
      elif animal_checker not in filename:
        labels.append('dog') # else add dog to label

      animal_photo = load_img(path + f'/{filename}', target_size = (150, 150)) # standardize photo size + loads
      animal_photo = img_to_array(animal_photo) # creates an array

      images.append(animal_photo) # append the photo to the images. The images list contains a list of arrays
    
    return asarray(images), asarray(labels), container; # not interested in list of arrays, but array containing lists. Asarray does this. Returns three arrays

In [3]:
train_data, train_labels, train_container = load_image_function('./train') # load train data

In [4]:
print(f'The training labels is: {train_labels[:5]} and the imported is {train_container[:5]}') # check train data is correct loaded

The training labels is: ['cat' 'cat' 'cat' 'cat' 'cat'] and the imported is ['cat.1500.jpg', 'cat.1501.jpg', 'cat.1502.jpg', 'cat.1503.jpg', 'cat.1504.jpg']


In [5]:
test_data, test_labels, test_container = load_image_function('./test') # load test data

print(f'The test labels is: {test_labels[:5]} and the imported is {test_container[:5]}') # check test data is correct loaded

The test labels is: ['cat' 'cat' 'cat' 'cat' 'cat'] and the imported is ['cat.0.jpg', 'cat.1.jpg', 'cat.10.jpg', 'cat.100.jpg', 'cat.101.jpg']


In [6]:
val_data, val_labels, val_container = load_image_function('./validation') # load val data

print(f'The val labels is: {val_labels[:5]} and the imported is {val_container[:5]}') # check val data is correct loaded

The val labels is: ['cat' 'cat' 'cat' 'cat' 'cat'] and the imported is ['cat.230.jpg', 'cat.231.jpg', 'cat.232.jpg', 'cat.233.jpg', 'cat.234.jpg']


In [7]:
# checking if reshape is necessary. 

print(f'Train has shape: {train_data.shape} \nTest has shape: {test_data.shape} \nValidation has shape: {val_data.shape}')

# Since they all have (# rows, width, height, # of color channelse) reshaping is not necessary 

Train has shape: (1664, 150, 150, 3) 
Test has shape: (520, 150, 150, 3) 
Validation has shape: (416, 150, 150, 3)


# Task 2 - Create the datagenerators

Due to the high amount of data, you need to write a datagenerator to load the images. This requires you to use the _ImageDataGenerator_ (as shown in Keras Intro - 2) in which you must apply at least one on-the-fly data augmentation. Which type of data augmentation is up to you, but you need to justify your choice in your report. On top of the _ImagaDataGenerator_ you need to apply the _.flow_from_directory_ function to make it work with the directory tree in Task 1

We need to use (at least) one on-the-fly augmentation. Which is better to use? Zooming, flipping, shifting, changing brightness? Should we use more than one? 

ImageDataGenerator: 
https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
flow_from_directory: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator#flow_from_directory

In [17]:
os.getcwd()

'C:\\Users\\45306\\Documents\\GitHub\\Deep_learning_Data'

In [9]:
# Image Data Generator -- (From Intro to Keras 2, the default settings)
data_gen = ImageDataGenerator(
        featurewise_center=False, samplewise_center=False,
        featurewise_std_normalization=False, samplewise_std_normalization=False,
        zca_whitening=False, zca_epsilon=1e-06, rotation_range=0, width_shift_range=0.0,
        height_shift_range=0.0, brightness_range=None, shear_range=0.0, zoom_range=0.0,
        channel_shift_range=0.0, fill_mode='nearest', cval=0.0,
        horizontal_flip=False, vertical_flip=False, rescale=None,
        preprocessing_function=None, data_format=None, validation_split=0.0, dtype=None
    )

In [21]:
# Taken from this site: https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html 
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [23]:
# Taken from the same site. Here, we only take one image, and puts it in the "preview"-directory, which I have made manually. 
# Problem: It does not delete the pictures when running the chunk again. 
img = load_img('Train/cat.871.jpg')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='cat', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely